In [39]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [40]:
import json

class JsonModelParser:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = self.load_json_file()
        self.parse_model_data()

    def printDetails(self):
        print("\tINPUT SIZE:",self.input_size)
        print("\tLAYERS:", self.layers)
        print("\tINPUT:", self.input)
        print("\tINITIAL WEIGHTS:", self.initial_weights)
        print("\tTARGET:", self.target)
        print("\tLEARNING RATE:", self.learning_rate)
        print("\tBATCH SIZE:", self.batch_size)
        print("\tMAX ITERATION:", self.max_iteration)
        print("\tERROR THRESHOLD:", self.error_threshold)
    

    def load_json_file(self):
        try:
            with open(self.filepath, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            print(f"The file {self.filepath} was not found")
            return None
        except json.JSONDecodeError:
            print(f"Error decoding JSON from the file {self.filepath}")
            return None

    def parse_model_data(self):
        if self.data:
            self.case = self.data.get('case', {})
            self.model = self.case.get('model', {})
            self.input_size = self.model.get('input_size')

            raw_layers = self.model.get('layers', [])   
            self.layers = [{'number_of_neurons': layer.get('number_of_neurons'),
                        'activation_function': layer.get('activation_function')}
                       for layer in raw_layers]
            
            self.input = self.case.get('input', [])
            print(self.case.get("initial_weights"))
            self.initial_weights = self.case.get('initial_weights', [])
            self.target = self.case.get('target', [])
            self.parameters = self.case.get('learning_parameters', {})

            self.learning_rate = self.parameters.get('learning_rate')
            self.batch_size = self.parameters.get('batch_size')
            self.max_iteration = self.parameters.get('max_iteration')
            self.error_threshold = self.parameters.get('error_threshold')

            self.expect = self.data.get('expect', {})
            self.stopped_by = self.expect.get('stopped_by', '')
            self.final_weights = self.expect.get('final_weights', [])

    @staticmethod
    def save_json_file(data, filepath):
        try:
            with open(filepath, 'w', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4)
        except IOError:
            print(f"Could not save data to {filepath}")

In [44]:
import tensorflow as tf
import numpy as np

class TensorFlowModel: 
    def __init__(self, inputs, targets, learning_rate, batch_size, initial_weights, layers) -> None:
        self.inputs = inputs
        self.targets = targets
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.input_size = len(inputs[0])
        self.initial_weights = initial_weights
        self.dataset = tf.data.Dataset.from_tensor_slices((self.inputs, self.targets))
        self.dataset = self.dataset.batch(batch_size)
        self.model = tf.keras.Sequential()
        
        for i, layer in enumerate(layers):
            if i == 0:
                self.model.add(tf.keras.layers.Dense(
                    layer['number_of_neurons'],
                    activation=layer['activation_function'],
                    input_shape=(self.input_size,),
                    kernel_initializer=lambda shape, dtype: tf.constant_initializer(initial_weights[i][1:])(shape, dtype=dtype),
                    bias_initializer=lambda shape, dtype: tf.constant_initializer(initial_weights[i][0])(shape, dtype=dtype)
                ))
            else:
                self.model.add(tf.keras.layers.Dense(
                    layer['number_of_neurons'],
                    activation=layer['activation_function'],
                    kernel_initializer=lambda shape, dtype: tf.constant_initializer(initial_weights[i][1:])(shape, dtype=dtype),
                    bias_initializer=lambda shape, dtype: tf.constant_initializer(initial_weights[i][0])(shape, dtype=dtype)
                ))
        
        loss_function = tf.keras.losses.CategoricalCrossentropy() if layers[-1]['activation_function'] == 'softmax' else tf.keras.losses.MeanSquaredError()
        optimizer = tf.keras.optimizers.SGD(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

    def fit_model(self, max_epochs, error_threshold):
        class ThresholdCallback(tf.keras.callbacks.Callback):
            def on_epoch_end(self, _, logs = None):
                if(logs.get("loss") < error_threshold):
                    self.model.stop_training = True
        thressholdCallback = ThresholdCallback()
        self.model.fit(self.dataset, epochs=max_epochs, callbacks=[thressholdCallback])
    def predict(self, new_input):
        # Ensure that new_input is in the correct shape (batch_size, input_size)
        new_input = np.array(new_input).reshape(1, -1)  # Reshape if one sample
        predictions = self.model.predict(new_input)
        return predictions
    
    def show_prediction(self):
        total_weights = []
        total_biases = []
        print("============================= Result =============================\n")
        if(self.model.stop_training):
            print(f"Stopped by : error_threshold\n")
        else:
            print("Stopped by : max_iteration\n")
        for i,layer in enumerate(self.model.layers):
            print(layer.get_weights())
            weights, biases = layer.get_weights()
            total_weights.append(weights)
            total_biases.append(biases)
            print(f"Layer-{i}")
            print(f"Weights : {weights}\n")
            print(f"Biases : {biases}\n")
        print("==================================================================")
        return total_weights, total_biases


In [42]:
import os

def main():
    currDir = os.getcwd()
    testDir = currDir.replace("src","test")
    file = os.path.join(testDir, "relu_b.json")
    json_file = JsonModelParser(file)
    inputs = json_file.input
    print("ini inputs : ", inputs)
    targets = json_file.target
    learning_rate = json_file.learning_rate
    layers = json_file.layers
    batch_size = json_file.batch_size
    initial_weights = json_file.initial_weights
    max_iter = json_file.max_iteration
    error_threshold = json_file.error_threshold
    model = TensorFlowModel(inputs, targets, learning_rate, batch_size, initial_weights, layers)
    model.fit_model(max_iter, error_threshold)
    model.show_prediction()
    
main()

[[[-0.2, 0.2, 1.0], [0.3, 0.5, 0.5], [-0.5, -1.0, 0.5]]]
ini inputs :  [[1.0, 0.8], [-0.3, -1.0]]


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - accuracy: 0.0000e+00 - loss: 0.9312
============================= Result =============================

Stopped by : max_iteration

[array([[ 0.30110002,  0.5095    ,  0.4335    ],
       [-0.49633333, -0.9683333 ,  0.43033332]], dtype=float32), array([-0.20366667,  0.16833334,  0.96166664], dtype=float32)]
Layer-0
Weights : [[ 0.30110002  0.5095      0.4335    ]
 [-0.49633333 -0.9683333   0.43033332]]

Biases : [-0.20366667  0.16833334  0.96166664]



In [45]:
import os
import csv

def main():
    currDir = os.getcwd()
    file = os.path.join(currDir, "model_data.json")
    json_file = JsonModelParser(file)
    inputs = json_file.input
    print("ini inputs : ", inputs)
    targets = json_file.target
    learning_rate = json_file.learning_rate
    layers = json_file.layers
    batch_size = json_file.batch_size
    initial_weights = json_file.initial_weights
    max_iter = json_file.max_iteration
    error_threshold = json_file.error_threshold
    model = TensorFlowModel(inputs, targets, learning_rate, batch_size, initial_weights, layers)
    model.fit_model(max_iter, error_threshold)
    weights, biases = model.show_prediction()

    with open("iris.csv","r") as csvfile : 
        datareader = csv.DictReader(csvfile)
        for row in datareader : 
            current = [
                float(row["SepalLengthCm"]),
                float(row["SepalWidthCm"]),
                float(row["PetalLengthCm"]),
                float(row["PetalWidthCm"])
            ]
            species = row["Species"]
            predictions = model.predict(current)
            print("Ini predictions :", predictions)
            break
    print(f"Ini weights : {weights}\n Ini biases : {biases}")
main()

[[[0.1, 0.3, 0.2], [0.4, 0.2, -0.7], [0.1, -0.8, 0.5], [0.1, 0.3, 0.2], [0.4, 0.2, -0.7]]]
ini inputs :  [[5.1, 3.5, 1.4, 0.2], [4.9, 3.0, 1.4, 0.2], [4.7, 3.2, 1.3, 0.2], [4.6, 3.1, 1.5, 0.2], [5.0, 3.6, 1.4, 0.2], [5.4, 3.9, 1.7, 0.4], [4.6, 3.4, 1.4, 0.3], [5.0, 3.4, 1.5, 0.2], [4.4, 2.9, 1.4, 0.2], [4.9, 3.1, 1.5, 0.1], [5.4, 3.7, 1.5, 0.2], [4.8, 3.4, 1.6, 0.2], [4.8, 3.0, 1.4, 0.1], [4.3, 3.0, 1.1, 0.1], [5.8, 4.0, 1.2, 0.2], [5.7, 4.4, 1.5, 0.4], [5.4, 3.9, 1.3, 0.4], [5.1, 3.5, 1.4, 0.3], [5.7, 3.8, 1.7, 0.3], [5.1, 3.8, 1.5, 0.3], [5.4, 3.4, 1.7, 0.2], [5.1, 3.7, 1.5, 0.4], [4.6, 3.6, 1.0, 0.2], [5.1, 3.3, 1.7, 0.5], [4.8, 3.4, 1.9, 0.2], [5.0, 3.0, 1.6, 0.2], [5.0, 3.4, 1.6, 0.4], [5.2, 3.5, 1.5, 0.2], [5.2, 3.4, 1.4, 0.2], [4.7, 3.2, 1.6, 0.2], [4.8, 3.1, 1.6, 0.2], [5.4, 3.4, 1.5, 0.4], [5.2, 4.1, 1.5, 0.1], [5.5, 4.2, 1.4, 0.2], [4.9, 3.1, 1.5, 0.1], [5.0, 3.2, 1.2, 0.2], [5.5, 3.5, 1.3, 0.2], [4.9, 3.1, 1.5, 0.1], [4.4, 3.0, 1.3, 0.2], [5.1, 3.4, 1.5, 0.2], [5.0, 3.5, 1.3

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7264 - loss: 1.4353 
============================= Result =============================

Stopped by : max_iteration

[array([[ 0.05775671,  0.14111806, -0.38728544],
       [-0.08108397, -0.8101177 ,  0.661761  ],
       [-0.11003511,  0.21955857,  0.39927277],
       [ 0.33552825,  0.16913599, -0.6366245 ]], dtype=float32), array([0.04086624, 0.29377073, 0.25311777], dtype=float32)]
Layer-0
Weights : [[ 0.05775671  0.14111806 -0.38728544]
 [-0.08108397 -0.8101177   0.661761  ]
 [-0.11003511  0.21955857  0.39927277]
 [ 0.33552825  0.16913599 -0.6366245 ]]

Biases : [0.04086624 0.29377073 0.25311777]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Ini predictions : [[-0.0353119 -1.4807298  1.0257825]]
Ini weights : [array([[ 0.05775671,  0.14111806, -0.38728544],
       [-0.08108397, -0.8101177 ,  0.661761  ],
       [-0.11003511,  0.21955857,  0.39927277],
       [ 0.33552825,  0.16913599, -0.6366245 ]], dtype=float32)]
 Ini biases : [array([